# Exercise 1


In [0]:
import pandas as pd
df1 = pd.read_csv('http://www.ntu.edu.sg/home/fscheong/public/citycolors.csv')
df1

,city,black,white,yellow,green,red,orange,purple
0,NY,2,13,2,14,7,12,7
1,SF,10,5,15,25,1,19,30
2,LA,77,23,12,34,45,56,65
3,DC,30,30,10,13,24,26,1


In [0]:
def find_max(row):
  max_value=row.max()
  comparison_table=row.apply(lambda x: x==max_value)
  return row.index[[i for i, true in enumerate(comparison_table) if true]].tolist()

df1.set_index('city',inplace=True)
df2=pd.DataFrame()
for i,row in df1.iterrows():
  max_cols=find_max(df1.loc[i])
  for col in max_cols:
    df2=df2.append({'city':i,'maxcol':col},ignore_index=True)    
df2

,city,maxcol
0,NY,green
1,SF,purple
2,LA,black
3,DC,black
4,DC,white


# Exercise 2

In [0]:
import pandas as pd
df1 = pd.read_stata('http://www.stata-press.com/data/r11/auto.dta')
df1=df1.dropna()
df=df1[['rep78','foreign','mpg','weight']]

In [0]:
from sklearn.linear_model import LinearRegression

result=pd.DataFrame()
result.at['All','All']=(LinearRegression().fit(df[['weight']], df[['mpg']]).coef_)
#partition into groups
foreign_values=df['foreign'].unique().tolist()
for value in foreign_values:
    subset=df.loc[df['foreign'] == value].copy()
    result.at['All',value]=(LinearRegression().fit(subset[['weight']], subset[['mpg']]).coef_)
    
rep78_values=df['rep78'].unique().tolist()
for value in rep78_values:
    subset=df.loc[df['rep78'] == value].copy()
    result.at[value,'All']=(LinearRegression().fit(subset[['weight']], subset[['mpg']]).coef_)
    
    for foreign_value in subset['foreign'].unique():
      sub_subset=subset.loc[subset['foreign'] == foreign_value].copy()
      result.at[value,foreign_value]=(LinearRegression().fit(sub_subset[['weight']], sub_subset[['mpg']]).coef_)

In [0]:
result['rep78']=result.index.astype(str)
result.sort_values('rep78')[['rep78','Domestic','Foreign','All']].reset_index(drop=True)

,rep78,Domestic,Foreign,All
0,1.0,-0.008108,NaN,-0.008108
1,2.0,-0.008046,NaN,-0.008046
2,3.0,-0.005066,-0.015476,-0.004410
3,4.0,-0.005002,-0.005500,-0.004957
4,5.0,-0.012500,-0.019698,-0.018160
5,All,-0.005999,-0.010747,-0.005960


# Exercise 3

In [0]:
import pandas as pd
df1 = pd.read_stata('http://www.stata-press.com/data/r11/reshape1.dta')
df1

,id,sex,inc80,inc81,inc82,ue80,ue81,ue82
0,1.0,0.0,5000.0,5500.0,6000.0,0.0,1.0,0.0
1,2.0,1.0,2000.0,2200.0,3300.0,1.0,0.0,0.0
2,3.0,0.0,3000.0,2000.0,1000.0,0.0,0.0,1.0


In [0]:
#stack()
#add another level of index
import re
lst=[]
df2=df1.set_index(['id','sex'])
for col in df2.columns: lst.append(re.findall(r'(\w+?)(\d+)',col)[0] )
df2.columns=pd.MultiIndex.from_tuples(lst)
df2.stack()

inc   ue
id  sex                
1.0 0.0 80  5000.0  0.0
        81  5500.0  1.0
        82  6000.0  0.0
2.0 1.0 80  2000.0  1.0
        81  2200.0  0.0
        82  3300.0  0.0
3.0 0.0 80  3000.0  0.0
        81  2000.0  0.0
        82  1000.0  1.0

In [0]:
#melt()
df_melt=df1
df_melt_inc=pd.melt(df_melt,id_vars=['id','sex'],value_vars =['inc80','inc81','inc82'], var_name='year', value_name="inc").replace({'year':{'inc80':80, 'inc81': 81, 'inc82': 82}})
df_melt_ue=pd.melt(df_melt,id_vars=['id','sex'],value_vars =['ue80','ue81','ue82'], var_name='year', value_name="ue").replace({'year':{'ue80':80, 'ue81': 81, 'ue82': 82}})
pd.merge(df_melt_inc,df_melt_ue, left_on=['id','sex','year'],right_on=['id','sex','year']).sort_values(by=['id','year','ue'], axis=0).reset_index(drop=True)

,id,sex,year,inc,ue
0,1.0,0.0,80,5000.0,0.0
1,1.0,0.0,81,5500.0,1.0
2,1.0,0.0,82,6000.0,0.0
3,2.0,1.0,80,2000.0,1.0
4,2.0,1.0,81,2200.0,0.0
5,2.0,1.0,82,3300.0,0.0
6,3.0,0.0,80,3000.0,0.0
7,3.0,0.0,81,2000.0,0.0
8,3.0,0.0,82,1000.0,1.0


In [0]:
#using nested for loops
new_df=pd.DataFrame()
years=[80,81,82]
for i,row in df1.iterrows():
   for j in range (0,3):
        new_df=new_df.append({'id':row.loc['id'],'sex':row.loc['sex'],'year':years[j],'inc':row.iloc[2+j],'ue':row.iloc[5+j]},ignore_index=True)
new_df

,id,inc,sex,ue,year
0,1.0,5000.0,0.0,0.0,80.0
1,1.0,5500.0,0.0,1.0,81.0
2,1.0,6000.0,0.0,0.0,82.0
3,2.0,2000.0,1.0,1.0,80.0
4,2.0,2200.0,1.0,0.0,81.0
5,2.0,3300.0,1.0,0.0,82.0
6,3.0,3000.0,0.0,0.0,80.0
7,3.0,2000.0,0.0,0.0,81.0
8,3.0,1000.0,0.0,1.0,82.0
